In [18]:
from pysc2.agents import base_agent
from pysc2.env import sc2_env
from pysc2.lib import actions, features, units
from absl import app
from absl import flags
import random

FLAGS = flags.FLAGS
flags.DEFINE_string('f', '', 'kernel')
class ZergAgent(base_agent.BaseAgent):
    
    def __init__(self):
        super(ZergAgent,self).__init__()

        self.attack_coordinates = None


    def can_do(self, obs, action):
        return action in obs.observation.available_actions
    def unit_type_is_selected(self, obs, unit_type):
        if (len(obs.observation.single_select) > 0 and
            obs.observation.single_select[0].unit_type == unit_type):
            return True
    
        if (len(obs.observation.multi_select) > 0 and
            obs.observation.multi_select[0].unit_type == unit_type):
            return True
    
        return False
 
    def get_units_by_type(self, obs, unit_type):
        return [unit for unit in obs.observation.feature_units if unit.unit_type == unit_type]

    def step(self, obs):
        super(ZergAgent, self).step(obs)
        ##init_enemy pos
        if obs.first():
            player_y, player_x = (obs.observation.feature_minimap.player_relative == features.PlayerRelative.SELF).nonzero()

            xmean = player_x.mean()
            ymean = player_y.mean()

            if xmean <= 31 and ymean <= 31:
                self.attack_coordinates = (49, 49)
            else:
                self.attack_coordinates = (12, 16)

        zerglings = self.get_units_by_type(obs,units.Zerg.Zergling)
#Attack with Zergling
        if(len(zerglings) > 20):
            
            if self.unit_type_is_selected(obs, units.Zerg.Zergling):
                if self.can_do(obs,actions.FUNCTIONS.Attack_minimap.id):
                    return actions.FUNCTIONS.Attack_minimap("now",self.attack_coordinates)
                if self.can_do(obs,actions.FUNCTIONS.select_army.id):
                    return actions.FUNCTIONS.select_army("select")
            
#Create Spawning_pool        
        spawning_pools = self.get_units_by_type(obs, units.Zerg.SpawningPool)
        if len(spawning_pools) == 0:
            if self.unit_type_is_selected(obs, units.Zerg.Drone):
                if (self.can_do(obs,actions.FUNCTIONS.Build_SpawningPool_screen.id)):
                    x = random.randint(0, 83)
                    y = random.randint(0, 83)
          
                    return actions.FUNCTIONS.Build_SpawningPool_screen("now", (x, y))

            drones = self.get_units_by_type(obs, units.Zerg.Drone)
            if len(drones) > 0:
                drone = random.choice(drones)

                return actions.FUNCTIONS.select_point("select_all_type", (drone.x,drone.y))  
#Create Overlord if no supply            
        if self.unit_type_is_selected(obs, units.Zerg.Larva):
              free_supply = (obs.observation.player.food_cap - obs.observation.player.food_used)
              if free_supply == 0:
                if (self.can_do(obs,actions.FUNCTIONS.Train_Overlord_quick.id)):
                    return actions.FUNCTIONS.Train_Overlord_quick("now")
#Train zergling
              if (self.can_do(obs,actions.FUNCTIONS.Train_Zergling_quick.id)):
                    return actions.FUNCTIONS.Train_Zergling_quick("now")

        larvae = self.get_units_by_type(obs, units.Zerg.Larva)
        
        if len(larvae) > 0:
            larva = random.choice(larvae)
      
            return actions.FUNCTIONS.select_point("select_all_type", (larva.x,larva.y))
        return actions.FUNCTIONS.no_op()

def main(unused_argv):
    agent = ZergAgent()
    try:
        with sc2_env.SC2Env(
                map_name="AbyssalReef",
        players=[sc2_env.Agent(sc2_env.Race.zerg), 
            sc2_env.Bot(sc2_env.Race.random,
            sc2_env.Difficulty.very_hard)],
            agent_interface_format=features.AgentInterfaceFormat(
            feature_dimensions=features.Dimensions(screen=84, minimap=64),use_feature_units=True),
            step_mul=16,
            game_steps_per_episode=0,           
            visualize=True) as env:            
            agent.setup(env.observation_spec(), env.action_spec())

            timesteps = env.reset()
            agent.reset()
            while True:
                step_actions = [agent.step(timesteps[0])]
                if timesteps[0].last():
                    break
                timesteps = env.step(step_actions)
    except KeyboardInterrupt:
        env.


In [19]:
if __name__ == "__main__":
    app.run(main)

I0610 13:39:00.204795 140251766867776 sc_process.py:110] Launching SC2: /home/dan/StarCraftII/Versions/Base70154/SC2_x64 -listen 127.0.0.1 -port 18534 -dataDir /home/dan/StarCraftII/ -tempDir /tmp/sc-8pyei836/ -displayMode 0 -windowwidth 640 -windowheight 480 -windowx 50 -windowy 50
I0610 13:39:00.215320 140251766867776 remote_controller.py:163] Connecting to: ws://127.0.0.1:18534/sc2api, attempt: 0, running: True
I0610 13:39:01.220617 140251766867776 remote_controller.py:163] Connecting to: ws://127.0.0.1:18534/sc2api, attempt: 1, running: True
I0610 13:39:02.224545 140251766867776 remote_controller.py:163] Connecting to: ws://127.0.0.1:18534/sc2api, attempt: 2, running: True
I0610 13:39:11.969290 140251766867776 sc2_env.py:322] Environment is ready on map: AbyssalReef
I0610 13:39:11.975306 140251766867776 sc2_env.py:462] Starting episode: 1
/home/dan/anaconda3/envs/StarcraftEnv/lib/python3.7/site-packages/pysc2/lib/features.py:862: FutureWarning: arrays to stack must be passed as a "

KeyboardInterrupt: Quit?

In [6]:


%tb

SystemExit: 1